<a href="https://colab.research.google.com/github/RCortez25/PhD/blob/main/LLM/4.%20Attention%20mechanism/3_Multi_head_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multihead attention wrapper

> In Multi head attention, one divides the attention mechanism into multiple heads, each operating independently.
>
> One stacks multiple single head attention layers. That is, one creates multiple instances of the self-attention mechanism, each with its own weights and then they are combined.
>
> Even though this is computationally expensive, it allows the LLM to capture complex patterns.

In [2]:
import torch
import torch.nn as nn

In [3]:
class MultiHeadAttentionWrapper(nn.Module):
    # Number of attention heads mut be given. These are the number of single
    # self-attention mechanisms
    def __init__(self, dimensions_in, dimensions_out, context_length,
                 dropout, num_heads, qkv_bias=False):
        super().__init__()
        # Create a list containing all single self-attention mechanisms. These
        # are created using the CausalAttention class
        self.heads = nn.ModuleList(
            [CausalAttention(dimensions_in, dimensions_out, context_length,
                             dropout, qkv_bias) for _ in range(num_heads)]
        )

    # Method for concatenating all single self-attention heads along the
    # columns
    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

> Let's repeat the code we had before to test the class

In [4]:
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your     (x^1)
     [0.55, 0.87, 0.66], # journey  (x^2)
     [0.57, 0.85, 0.64], # starts   (x^3)
     [0.22, 0.58, 0.33], # with     (x^4)
     [0.77, 0.25, 0.10], # one      (x^5)
     [0.05, 0.80, 0.55]] # step     (x^6)
)

batch = torch.stack([inputs, inputs], dim=0)

In [5]:
class CausalAttention(nn.Module):
    def __init__(self, dimension_inputs, dimension_outputs, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.dimension_outputs = dimension_outputs
        # Initialize the matrices using Linear layers
        self.W_q = nn.Linear(dimension_inputs, dimension_outputs, bias=qkv_bias)
        self.W_k = nn.Linear(dimension_inputs, dimension_outputs, bias=qkv_bias)
        self.W_v = nn.Linear(dimension_inputs, dimension_outputs, bias=qkv_bias)
        # Initialize the dropout layer
        self.dropout = nn.Dropout(dropout)
        # Initialize the buffer, for automatically moving the model to CPU or GPU
        self.register_buffer("mask",
                             torch.triu(torch.ones(context_length, context_length),
                                        diagonal=1))

    # Method to calculate the context vector
    def forward(self, input_vectors):
        # Obtain the relevant dimensions
        batch_size, number_of_tokens, dimension_inputs = input_vectors.shape
        queries = self.W_q(input_vectors)
        keys = self.W_k(input_vectors)
        values = self.W_v(input_vectors)

        attention_scores = queries @ keys.transpose(1, 2)
        attention_scores.masked_fill_(
            self.mask.bool()[:number_of_tokens, :number_of_tokens], -torch.inf)

        # Calculate attention weights
        dimension_keys = keys.shape[-1]
        attention_weights = torch.softmax(attention_scores / (dimension_keys ** 0.5), dim=-1)
        # Apply dropout
        attention_weights = self.dropout(attention_weights)

        # Calculate and return the context vectors
        context_vectors = attention_weights @ values

        return context_vectors

In [6]:
# Test the wrapper
torch.manual_seed(123)
context_length = batch.shape[1] # Number of tokens
dimension_inputs = 3
dimension_outputs = 2
oMultiHeadAttention = MultiHeadAttentionWrapper(dimension_inputs,
                                                dimension_outputs,
                                                context_length,
                                                dropout=0.0, num_heads=2)
context_vectors = oMultiHeadAttention(batch)
print(context_vectors)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)


In [7]:
context_vectors.shape

torch.Size([2, 6, 4])

> The dimensions here represent:
* 2: The number of batches
* 6: The number of tokens, or input vectors, in each batch
* 4: Ths is because we selected two heads `num_heads=2`. Therefore, since the output dimension of each head is 2, then 2 heads add 2 + 2 = 4.

# Multi-head attention with weight splits

> Now, the problem with the wrapper is that is not very efficient because of the matrix multiplications involved. We multiply the inputs by two matrices (`num_heads=2`). We can achieve the same result by performing only one matrix multiplication.Let's combine the two classes CausalAttention and MultiHeadAttentionWrapper together and add some other modifications.

In [8]:
class MultiHeadAttention(nn.Module):
    def __init__(self, dimension_inputs, dimension_outputs, context_length,
                 dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (dimension_outputs % num_heads == 0), "Dimension outputs must be \
                                                    divisible by number of heads" # 1
        self.dimension_outputs = dimension_outputs
        self.num_heads = num_heads
        self.head_dimension = dimension_outputs // num_heads # 2

        #Initialize weight matrices
        self.W_query = nn.Linear(dimension_inputs, dimension_outputs, bias=qkv_bias)
        self.W_key = nn.Linear(dimension_inputs, dimension_outputs, bias=qkv_bias)
        self.W_value = nn.Linear(dimension_inputs, dimension_outputs, bias=qkv_bias)
        # Initialize a linear layer to combine head outputs
        self.output_projection = nn.Linear(dimension_outputs, dimension_outputs) # 3
        # Initialize the dropout layer
        self.dropout = nn.Dropout(dropout)
        self.register_buffer("mask",
                             torch.triu(torch.ones(context_length, context_length),
                                        diagonal=1))

    # Method for calculating the context vectors
    def forward(self, input_vectors):
        batch_size, number_of_tokens, dimension_inputs = input_vectors.shape

        # Calculate keys, queries, and values
        queries = self.W_query(input_vectors)
        keys = self.W_key(input_vectors)
        values = self.W_value(input_vectors)

        # Split the obtained matrices by adding a "number of heads" dimension
        # that is, the output dimension will be splitted as
        # (batch_size, number_of_tokens, dimension_outputs) will be
        # (batch_size, number_of_tokens, num_heads, head_dimension)
        keys = keys.view(batch_size, number_of_tokens, self.num_heads, self.head_dimension)
        values = values.view(batch_size, number_of_tokens, self.num_heads, self.head_dimension)
        queries = queries.view(batch_size, number_of_tokens, self.num_heads, self.head_dimension)

        # Transpose for performing calculations
        keys = keys.transpose(1, 2)
        values = values.transpose(1, 2)
        queries = queries.transpose(1, 2)

        # Calculate attention scores
        attention_scores = queries @ keys.transpose(2, 3)

        # TO-DO
        attention_scores.masked_fill_(
            self.mask.bool()[:number_of_tokens, :number_of_tokens], -torch.inf)

        # Calculate attention

> Comments:
>
> * 1: Use the `assert` to ensure that dimension is correct
* 2: Calculate the dimension of each head
* 3:  

# Simple example of multihead attention

## Step 1: Inputs

Say we have 1 batch of 3 tokens, each in a 6-dimensional space. Say the input batch is "The cat sleeps". That is

`batch_size=1` \
`number_of_tokens=3` \
`dimension_inputs=6`

In [19]:
x = torch.tensor([[
    [1.0, 2.0, 3.0, 4.0, 5.0, 6.0], # The
    [6.0, 5.0, 4.0, 3.0, 2.0, 1.0], # cat
    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]  # sleeps
]])

## Step 2: Decide the output dimension and the number of heads

`dimension_outputs=6` \
`num_heads=2`

Recall the the output dimension is the dimension of the context vector for each input token. Typically, in GPT models `dimension_input=dimension_outputs`.

For GPT models, the number of heads is 96, but in this example we'll keep it simple.

This decision gives

`head_dimension = dimension_outputs/num_heads = 6/2 = 3`

## Spet 3: Initialize trainable weight matrices

Initialize `W_query, W_key_, W_value`. Their dimension must be

$$dimension\_inputs\times dimension\_outputs$$

so, in this case one has

$$6\times6$$

this is done so in order to make matrix multiplication plausible.

In [15]:
W_query = nn.Linear(6, 6)
W_key = nn.Linear(6, 6)
W_value = nn.Linear(6, 6)

print(W_query)
print(W_key)
print(W_value)

Linear(in_features=6, out_features=6, bias=True)
Linear(in_features=6, out_features=6, bias=True)
Linear(in_features=6, out_features=6, bias=True)


Print the actual values of the matrices. Note that they contain bias which in this case is not important.

In [16]:
print("W_query weights:\n", W_query.weight)
print("W_query bias:\n", W_query.bias)

print("\nW_key weights:\n", W_key.weight)
print("W_key bias:\n", W_key.bias)

print("\nW_value weights:\n", W_value.weight)
print("W_value bias:\n", W_value.bias)

W_query weights:
 Parameter containing:
tensor([[-0.2389,  0.3859, -0.1704,  0.0871, -0.1995, -0.1969],
        [ 0.1828, -0.1140, -0.2589, -0.1669,  0.2977,  0.2458],
        [ 0.2485, -0.3484,  0.1923,  0.1019, -0.2745,  0.0129],
        [ 0.0816, -0.2205, -0.1723,  0.3330, -0.0330, -0.0043],
        [-0.2584, -0.2441,  0.3759,  0.1519, -0.0646,  0.2487],
        [-0.1942, -0.3561, -0.4053,  0.1659, -0.1592,  0.2507]],
       requires_grad=True)
W_query bias:
 Parameter containing:
tensor([ 0.3488,  0.1345,  0.3508, -0.0940,  0.3558, -0.1947],
       requires_grad=True)

W_key weights:
 Parameter containing:
tensor([[-0.0536,  0.2713, -0.2115,  0.3115,  0.1001, -0.0080],
        [ 0.3494,  0.3062, -0.1679,  0.0396,  0.0476,  0.3344],
        [ 0.2294,  0.3306,  0.2489, -0.3552,  0.2712, -0.1084],
        [ 0.3275,  0.2569, -0.2386, -0.0430,  0.0609,  0.1167],
        [-0.3781,  0.0183,  0.2127,  0.2305,  0.2008,  0.0646],
        [-0.3916,  0.2686, -0.3215, -0.2399,  0.0048,  0.1243]

## Step 4: Calculate Keys, Queries, and Values matrices

$$inputs * W_{keys}$$
$$inputs * W_{queries}$$
$$inputs * W_{values}$$

The resulting dimensions will be

$$1\times3\times6$$

In [21]:
Keys = W_key(x)
print("Keys:\n", Keys)

Queries = W_query(x)
print("\nQueries:\n", Queries)

Values = W_value(x)
print("\nValues:\n", Values)

Keys:
 tensor([[[ 1.7901,  2.8035,  0.7118,  0.9231,  2.7435, -1.2384],
         [ 1.5530,  3.4468,  3.1830,  2.3697, -0.0352, -3.1080],
         [ 0.6470,  0.8520,  0.4063,  0.4453,  0.4825, -0.7850]]],
       grad_fn=<ViewBackward0>)

Queries:
 tensor([[[-1.4601,  1.6082, -0.4082,  0.1709,  2.5136, -0.9453],
         [-0.1715, -0.0334,  0.6383, -0.4673, -0.3364, -4.3318],
         [ 0.0160,  0.3210,  0.2834, -0.1095,  0.5651, -0.8929]]],
       grad_fn=<ViewBackward0>)

Values:
 tensor([[[-1.6174,  0.7487,  2.8835,  1.3917, -4.1323, -0.0449],
         [-2.5170, -0.6037,  5.2189, -2.5608,  0.6473,  2.7826],
         [-0.5924, -0.0637,  1.0161, -0.0302, -0.5233,  0.3333]]],
       grad_fn=<ViewBackward0>)


## Step 5: Unroll the last dimension of the Keys, Queries, Values matrices to include the number of heads and the dimension of heads.


(batch_size, number_of_tokens, dimension_outputs) ->
(batch_size, number_of_tokens, num_heads, head_dimension)

that is

(1, 3, 6) ->
(1, 3, 2, 3)


In [22]:
reshaped_queries = Queries.view(1, 3, 2, 3)
reshaped_keys = Keys.view(1, 3, 2, 3)
reshaped_values = Values.view(1, 3, 2, 3)

In [23]:
print("Reshaped queries:\n", reshaped_queries)
print("\nReshaped keys:\n", reshaped_keys)
print("\nReshaped values:\n", reshaped_values)

Reshaped queries:
 tensor([[[[-1.4601,  1.6082, -0.4082],
          [ 0.1709,  2.5136, -0.9453]],

         [[-0.1715, -0.0334,  0.6383],
          [-0.4673, -0.3364, -4.3318]],

         [[ 0.0160,  0.3210,  0.2834],
          [-0.1095,  0.5651, -0.8929]]]], grad_fn=<ViewBackward0>)

Reshaped keys:
 tensor([[[[ 1.7901,  2.8035,  0.7118],
          [ 0.9231,  2.7435, -1.2384]],

         [[ 1.5530,  3.4468,  3.1830],
          [ 2.3697, -0.0352, -3.1080]],

         [[ 0.6470,  0.8520,  0.4063],
          [ 0.4453,  0.4825, -0.7850]]]], grad_fn=<ViewBackward0>)

Reshaped values:
 tensor([[[[-1.6174,  0.7487,  2.8835],
          [ 1.3917, -4.1323, -0.0449]],

         [[-2.5170, -0.6037,  5.2189],
          [-2.5608,  0.6473,  2.7826]],

         [[-0.5924, -0.0637,  1.0161],
          [-0.0302, -0.5233,  0.3333]]]], grad_fn=<ViewBackward0>)


The first groud of two elements of each matrix corresponds to the first token, and each element corresponds to one head. For instance, for the reshaped queries matrix:

[-1.4601,  1.6082, -0.4082], \
[ 0.1709,  2.5136, -0.9453]

corresponds to the first token "The", and each row corresponds to one head each. Then

[-0.1715, -0.0334,  0.6383], \
[-0.4673, -0.3364, -4.3318]

corresponds to the second token "cat", and each row corresponds to one head each. And so on.

Lastly, each token is now in 3 dimensions, hence the 3 elements of each row, because the head dimension equals 3.

Recall then that each head pays attention to each token separatedly.

## Step 6: Group by heads

For performing calculations we need to reshape the matrices to be of the following format

(batch_size, number_of_tokens, num_heads, head_dimension) ->
(batch_size, num_heads, number_of_tokens, head_dimension)

that is

(1, 3, 2, 3) ->
(1, 2, 3, 3)

Now, given that we have indexes 0,1,2,3 and we want to interchange indexes 1 and 2, we transpose the matrix using these indexes.

In [26]:
transposed_queries = reshaped_queries.transpose(1, 2) # This means interchanging indexes 1 and 2
transposed_keys = reshaped_keys.transpose(1, 2)
transposed_values = reshaped_values.transpose(1, 2)

In [27]:
print("Transposed queries:\n", transposed_queries)
print("\nTransposed keys:\n", transposed_keys)
print("\nTransposed values:\n", transposed_values)
#

Transposed queries:
 tensor([[[[-1.4601,  1.6082, -0.4082],
          [-0.1715, -0.0334,  0.6383],
          [ 0.0160,  0.3210,  0.2834]],

         [[ 0.1709,  2.5136, -0.9453],
          [-0.4673, -0.3364, -4.3318],
          [-0.1095,  0.5651, -0.8929]]]], grad_fn=<TransposeBackward0>)

Transposed keys:
 tensor([[[[ 1.7901,  2.8035,  0.7118],
          [ 1.5530,  3.4468,  3.1830],
          [ 0.6470,  0.8520,  0.4063]],

         [[ 0.9231,  2.7435, -1.2384],
          [ 2.3697, -0.0352, -3.1080],
          [ 0.4453,  0.4825, -0.7850]]]], grad_fn=<TransposeBackward0>)

Transposed values:
 tensor([[[[-1.6174,  0.7487,  2.8835],
          [-2.5170, -0.6037,  5.2189],
          [-0.5924, -0.0637,  1.0161]],

         [[ 1.3917, -4.1323, -0.0449],
          [-2.5608,  0.6473,  2.7826],
          [-0.0302, -0.5233,  0.3333]]]], grad_fn=<TransposeBackward0>)


Now, in the result, the first element of each matrix corresponds to the 3 tokens for the first head, and the second element corresponds to the 3 tokens for the second head.

That is, for the queries matrix

[[-1.4601,  1.6082, -0.4082], \
[-0.1715, -0.0334,  0.6383], \
[ 0.0160,  0.3210,  0.2834]]

corresponds to the 3 tokens for the first head, and

[[ 0.1709,  2.5136, -0.9453], \
[-0.4673, -0.3364, -4.3318], \
[-0.1095,  0.5651, -0.8929]]

corresponds to the 3 tokens for the second head.

## Step 7: Find attention scores (self-attention)

$$queries * keys.transpose(2,3)$$

We need to transpose

In [29]:
attention_scores = transposed_queries @ transposed_keys.transpose(2, 3)
print(attention_scores)

tensor([[[[ 1.6042,  1.9762,  0.2595],
          [ 0.0538,  1.6504,  0.1200],
          [ 1.1305,  2.0336,  0.3990]],

         [[ 8.2246,  3.2545,  2.0309],
          [ 4.0104, 12.3678,  3.0300],
          [ 2.5552,  2.4958,  0.9248]]]], grad_fn=<UnsafeViewBackward0>)


Once again, each big block corresponds to each head. These block corresponds to attention scores. Recall that each row is a word and each column is also a word.

So the first element 1.6042 corresponds to the attention score between "The" and "The". The element 1.9762 is the attention score between "The" and "cat". The element 0.1200 corresponds to the attention score beteween "cat" and "sleeps". All these correspond to the first head and the same applies for the second big block which corresponds to the second head.

## Step 8: Calculate attention weights

Recall that in causal attention one masks future tokens. Firts, we mask future tokens, then we divide by the "scaled dot-product self-attention", then we apply softmax.

In [30]:
mask = torch.triu(torch.ones(attention_scores.size(-2), attention_scores.size(-1)), diagonal=1).bool()
attention_scores_masked = attention_scores.masked_fill(mask, -torch.inf)
print("Masked attention scores:\n", attention_scores_masked)

Masked attention scores:
 tensor([[[[ 1.6042,    -inf,    -inf],
          [ 0.0538,  1.6504,    -inf],
          [ 1.1305,  2.0336,  0.3990]],

         [[ 8.2246,    -inf,    -inf],
          [ 4.0104, 12.3678,    -inf],
          [ 2.5552,  2.4958,  0.9248]]]], grad_fn=<MaskedFillBackward0>)


In [31]:
head_dimension = transposed_keys.size(-1) # Get the head dimension
attention_scores_scaled = attention_scores_masked / (head_dimension ** 0.5)
print("Scaled attention scores:\n", attention_scores_scaled)

Scaled attention scores:
 tensor([[[[0.9262,   -inf,   -inf],
          [0.0311, 0.9529,   -inf],
          [0.6527, 1.1741, 0.2304]],

         [[4.7485,   -inf,   -inf],
          [2.3154, 7.1406,   -inf],
          [1.4752, 1.4410, 0.5339]]]], grad_fn=<DivBackward0>)


In [32]:
attention_weights = torch.softmax(attention_scores_scaled, dim=-1)
print("Attention weights:\n", attention_weights)

Attention weights:
 tensor([[[[1.0000, 0.0000, 0.0000],
          [0.2846, 0.7154, 0.0000],
          [0.2994, 0.5043, 0.1963]],

         [[1.0000, 0.0000, 0.0000],
          [0.0080, 0.9920, 0.0000],
          [0.4244, 0.4101, 0.1656]]]], grad_fn=<SoftmaxBackward0>)


After this, one could implement dropout, but for the sake of simplicity we will leave it as is.

## Step 9: